# Fun with Neural Nets

---

Below is a procedure for building a neural network to recognize handwritten digits.  The data is from Kaggle, and you will submit your results to Kaggle to test how well you did!

1. Load the training data (`train.csv`) from Kaggle
2. Setup X and y (feature matrix and target vector)
3. Split X and y into train and test subsets.
4. Preprocess your data

   - When dealing with image data, you need to normalize your `X` by dividing each value by the max value of a pixel (255).
   - Since this is a multiclass classification problem, keras needs `y` to be a one-hot encoded matrix
   
5. Create your network.

   - Remember that for multi-class classification you need a softamx activation function on the output layer.
   - You may want to consider using regularization or dropout to improve performance.
   
6. Trian your network.
7. If you are unhappy with your model performance, try to tighten up your model by adding hidden layers, adding hidden layer units, chaning the activation functions on the hidden layers, etc.
8. Load in Kaggle's `test.csv`
9. Create your predictions (these should be numbers in the range 0-9).
10. Save your predictions and submit them to Kaggle.

---

For this lab, you should complete the above sequence of steps for _at least_ two of the three "configurations":

1. Using a `tensorflow` network
2. Using a `keras` "sequential" network
3. Using a `keras` convolutional network
4. Using a `tensorflow` convolutional network (we did _not_ cover this in class!)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from keras import regularizers
from keras.utils import np_utils

%matplotlib inline

Using TensorFlow backend.


In [2]:
df = pd.read_csv('/Users/Arya/.kaggle/competitions/digit-recognizer/train.csv')

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
        df.drop(columns='label'), df['label'], test_size=.2)

In [4]:
y_train.value_counts()

1    3735
7    3508
3    3496
2    3357
0    3310
9    3308
4    3279
6    3272
8    3257
5    3078
Name: label, dtype: int64

## Convolutional Approach

In [6]:
# Rescale range from 0-255 to 0-1
X_train /= 255
X_test /= 255

# Preprocess input data - 1 because greyscale (3 if RGB)
X_train = X_train.values.reshape(-1, 28, 28, 1)
X_test = X_test.values.reshape(-1, 28, 28, 1)

# Typecast as float
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [7]:
# Make y categorical
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [8]:
# Define model architecture
model = Sequential()

model.add(Convolution2D(6, #6 filters
                        3, # kernel size (3x3 filter)
                        activation='relu',
                        input_shape=(28, 28, 1)
                        ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(16,
                        3,
                        activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation=('softmax')))

In [9]:
# Compile model
model.compile('adam',
              'categorical_crossentropy', # good for unordered discrete predictions
              ['accuracy'])

In [10]:
# Fit model
model.fit(X_train, y_train, 32, 15)

Epoch 1/15
33600/33600 [==============================] - 22s 647us/step - loss: 0.4467 - acc: 0.8564
Epoch 2/15
33600/33600 [==============================] - 22s 651us/step - loss: 0.1842 - acc: 0.9438
Epoch 3/15
33600/33600 [==============================] - 28s 845us/step - loss: 0.1388 - acc: 0.9567
Epoch 4/15
33600/33600 [==============================] - 29s 851us/step - loss: 0.1186 - acc: 0.9639
Epoch 5/15
33600/33600 [==============================] - 28s 840us/step - loss: 0.1049 - acc: 0.9676
Epoch 6/15
33600/33600 [==============================] - 23s 689us/step - loss: 0.0961 - acc: 0.9699
Epoch 7/15
33600/33600 [==============================] - 21s 636us/step - loss: 0.0917 - acc: 0.9706
Epoch 8/15
33600/33600 [==============================] - 22s 645us/step - loss: 0.0849 - acc: 0.9745
Epoch 9/15
33600/33600 [==============================] - 22s 646us/step - loss: 0.0798 - acc: 0.9759
Epoch 10/15
33600/33600 [==============================] - 22s 649us/step - loss: 

In [11]:
# Evaluate model
score = model.evaluate(X_test, y_test, verbose=1)
labels = model.metrics_names

print(str(labels[0]) + ': ' + str(score[0]))
print(str(labels[1]) + ': ' + str(score[1]))

8400/8400 [==============================] - 3s 339us/step
loss: 0.0411530549000045
acc: 0.9879761904761905


In [12]:
kagtest = pd.read_csv('/Users/Arya/.kaggle/competitions/digit-recognizer/test.csv')

In [13]:
# Establish parity
kagtest /= 255
kagtest = kagtest.values.reshape(-1, 28, 28, 1)
kagtest = X_test.astype('float32')

In [14]:
# predict results
results = model.predict(kagtest)

# select the index with the maximum probability
results = np.argmax(results, axis = 1)
results = pd.Series(results, name="Label")

In [15]:
results.value_counts()

1    949
7    891
9    891
6    868
3    843
0    829
2    828
8    803
4    791
5    707
Name: Label, dtype: int64

In [ ]:
#submission = pd.concat([pd.Series(range(1,28001), name = "ImageId"), results], axis = 1)

#submission.to_csv("convolutionalpreds.csv", index=False)

## Sequential Approach

In [170]:
# Reload data to undo reshaping
df = pd.read_csv('/Users/Arya/.kaggle/competitions/digit-recognizer/train.csv')
# Resplit
X_train, X_test, y_train, y_test = train_test_split(
        df.drop(columns='label'), df['label'], test_size=.2)
# Make y categorical
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [171]:
# Rescale range from 0-255 to 0-1
X_train /= 255
X_test /= 255

# Typecast as float
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [172]:
X_train.shape

(33600, 784)

In [173]:
y_train.shape

(33600, 10)

In [174]:
y_train.shape

(33600, 10)

In [175]:
model = Sequential()
model.add(Dense(X_train.shape[1], activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
#model.add(Dense(16800, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax')) # Activation = Softmax for classification

In [176]:
# Compile model
model.compile('adam',
              'categorical_crossentropy',
              ['accuracy'])

In [ ]:
# Fit model - but don't actually unless you want to melt your laptop
#model.fit(X_train, Y_train, 64, 15, validation_data=(X_test, y_test))

Train on 33600 samples, validate on 8400 samples
Epoch 1/15


In [ ]:
# Evaluate model
score = model.evaluate(X_test, y_test, verbose=1)
labels = model.metrics_names

print(str(labels[0]) + ': ' + str(score[0]))
print(str(labels[1]) + ': ' + str(score[1]))